# 1. Cel projektu

## 1. Cel i zakres projektu

### Cel
Głównym celem projektu jest analiza jakości powietrza w Polsce na podstawie danych dotyczących różnych wskaźników czystości powietrza (np. pyły PM2.5, PM10) zbieranych z różnych stacji pomiarowych na terenie kraju.

**Projekt ma na celu:**

    1. Identyfikację obszarów o najwyższym i najniższym poziomie zanieczyszczeń.

    2. Analizę trendów czasowych (np. sezonowych) w stężeniach zanieczyszczeń.

    3. Ocenę wpływu czynników takich jak lokalizacja stacji (miejskie, wiejskie), typ obszaru (przemysłowy, komunikacyjny).

    4. Dostarczenie informacji dla decydentów i społeczeństwa, które mogą przyczynić się do poprawy jakości powietrza.

### Zakres
#### Dane: 
    Wykorzystane zostaną dane z różnych stacji pomiarowych na terenie Polski, zawierające informacje o stężeniach pyłów PM2.5, PM10 oraz innych wskaźników jakości powietrza. Dane zostały pozyskane ze strony internetowej Głównego Inspektoratu Ochrony Środowiska (https://powietrze.gios.gov.pl).

#### Analiza: 
    Dane zostaną poddane analizie statystycznej i przestrzennej, w tym:
    
    - Mapowanie stężeń zanieczyszczeń.
    
    - Identyfikacja obszarów o najwyższym poziomie zanieczyszczeń.
    
    - Analiza trendów czasowych.
    
    - Korelacje z czynnikami takimi jak typ obszaru.

#### Wyniki: 
    Projekt dostarczy wizualizacje, które będą mogły być wykorzystane do podejmowania decyzji dotyczących ochrony środowiska i zdrowia publicznego.

## 2. Dataset

### Opis danych
Każdy rekord z rekordów jest uśrednionym pomiarem danego wskaźnika z 24 godzin, zrealizowanym w konkretnej lokalizacji.

| Nazwa kolumny       | Opis                                                                 |
|---------------------|---------------------------------------------------------------------|
| `kod_stacji`        | Unikalny kod identyfikujący stację pomiarową.                       |
| `wskaznik`          | Rodzaj mierzonego wskaźnika (np. PM2.5, PM10, NO2).                 |
| `czas_usredniania`  | Okres uśredniania pomiarów (np. 1 godzina, 24 godziny).             |
| `jednostka`         | Jednostka miary dla wskaźnika (np. µg/m³).                          |
| `data`              | Data pomiaru.                                                       |
| `pomiar`            | Zmierzona wartość wskaźnika.                                        |
| `stary_kod_stacji`  | Poprzedni kod stacji (jeśli istnieje).                              |
| `typ_obszaru`       | Typ obszaru, na którym znajduje się stacja (np. miejski, wiejski).  |
| `rodzaj_stacji`     | Rodzaj stacji pomiarowej (np. tło, komunikacyjna).                  |
| `wojewodztwo`       | Województwo, w którym znajduje się stacja.                          |
| `miejscowosc`       | Miejscowość, w której znajduje się stacja.                          |
| `latitude`          | Szerokość geograficzna stacji pomiarowej.                           |
| `longitude`         | Długość geograficzna stacji pomiarowej.                             |

Dataset został stworzony wstępnie w programie Excel (transpozycje, usunięcie zbędnych kolumn), a następnie przekształcony do potrzebnego nam formatu w pythonie za pomocą biblioteki pandas:
```python
import pandas as pd

def merge_measures_with_stations_data(df_data, df_stations_data, new_file_name: str, year: str):
    df_data_melted = df_data.melt(id_vars=['kod_stacji', 'wskaznik', 'czas_usredniania', 'jednostka'], var_name='data', value_name='pomiar')
    merged = pd.merge(df_data_melted, df_stations_data, on='kod_stacji', how='inner')
    merged['data'] = pd.to_datetime(merged['data'], format='%d-%m-%Y')
    merged.to_csv(f'data/{year}/{new_file_name}.csv', index=False)

df_stations = pd.read_excel('data/metadane_kody_stacji_i_stanowisk_pomiarowych.xlsx')
df_2023_pm_10 = pd.read_excel('data/2023/2023_PM10_24h_formatted.xlsx')
df_2022_pm_10 = pd.read_excel('data/2022/2022_PM10_24h_formatted.xlsx')
df_2021_pm_10 = pd.read_excel('data/2021/2021_PM10_24h_formatted.xlsx')

merge_measures_with_stations_data(df_2023_pm_10, df_stations, '2023_pm10_merged', '2023')
merge_measures_with_stations_data(df_2022_pm_10, df_stations, '2022_pm10_merged', '2022')
merge_measures_with_stations_data(df_2021_pm_10, df_stations, '2021_pm10_merged', '2021')
```

Dane geometryczne są w systemie odniesienia: **WGS84**, oparty na nim system przestrzenny to: **EPSG:4326**.

# 2. Wartość merytoryczna rozwiązania

### Wczytanie danych

In [4]:
import pandas as pd
import geopandas as gpd

In [6]:
df_2023_pm_10 = pd.read_csv('data/2023/2023_pm10_merged.csv') 
df_2022_pm_10 = pd.read_csv('data/2022/2022_pm10_merged.csv') 
df_2021_pm_10 = pd.read_csv('data/2021/2021_pm10_merged.csv')

In [7]:
geometry_2023 = gpd.points_from_xy(df_2023_pm_10['longitude'], df_2023_pm_10['latitude'])
geometry_2022 = gpd.points_from_xy(df_2022_pm_10['longitude'], df_2022_pm_10['latitude'])
geometry_2021 = gpd.points_from_xy(df_2021_pm_10['longitude'], df_2021_pm_10['latitude'])

In [8]:
gdf_2023_pm_10 = gpd.GeoDataFrame(df_2023_pm_10, geometry=geometry_2023)
gdf_2022_pm_10 = gpd.GeoDataFrame(df_2022_pm_10, geometry=geometry_2022)
gdf_2021_pm_10 = gpd.GeoDataFrame(df_2021_pm_10, geometry=geometry_2021)

In [11]:
gdf_2023_pm_10.set_crs(epsg=4326, inplace=True)
gdf_2022_pm_10.set_crs(epsg=4326, inplace=True)
gdf_2021_pm_10.set_crs(epsg=4326, inplace=True);

# 3. Wizualizacja rozwiązania